# Forest Vegetation Identification in British Columbia: A Static Algorithm Approach

Team: Green Guardian<br>
Members: Kay Cai, Dana Zhan, Mia Yan, Fengting Tang<br>
Mentors: Derek Jacoby, Yvonne Coady<br>

Notice: If you are viewing this notebook on GitHub, you may have noticed that all the outputs are cleared. This is due to the file size limiation when uploading to GitHub.
Please refer to [Google Colab Link](https://colab.research.google.com/drive/1mOwenrmkZ9UghsrzCILrqeOzSoji7eX-?usp=sharing) for the complete version.

## 1. Set Up

### 1-1. Set Up Libraries

In [ ]:
# install libraries
%pip install folium fsspec geopandas mapclassify python-dotenv pystac-client shapely opencv-python matplotlib tifffile rasterio pyproj

In [ ]:
# import libraries
from datetime import datetime
from pathlib import Path
import json
import os
import shutil
import requests
from dotenv import load_dotenv
from pystac_client import Client
from pystac.item import Item
from shapely.geometry import shape, Polygon, mapping
import copy
import cv2
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
import rasterio
from rasterio.mask import mask
from pyproj import Transformer
from PIL import Image
from IPython.display import display, HTML
from matplotlib.colors import Normalize
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from skimage.feature import graycomatrix, graycoprops
from skimage import data
from skimage.color import rgb2gray
import pandas as pd
from collections import defaultdict

### 1-2. Function Definitions

#### 1-2-1. Data Retrieving Functions

In [ ]:
'''
Function -- get_api_client
    Get client of the Earth Platform API to access sentinel data
Return:
    client -- client of the API
'''

def get_api_client():

    # load secrets
    load_dotenv(
        # You will need to create this file with your
        # CLIENT_ID, CLIENT_SECRET, AUTH_TOKEN_URL, and API_URL
        dotenv_path="secrets.env"
    )
    CLIENT_ID = os.environ.get("CLIENT_ID")
    CLIENT_SECRET = os.environ.get("CLIENT_SECRET")
    AUTH_TOKEN_URL = os.environ.get("AUTH_TOKEN_URL")
    API_URL = os.environ.get("API_URL")

    # authenticate with the Earth Platform and obtain a new access token
    token_req_payload = {"grant_type": "client_credentials"}
    token_response = requests.post(
        AUTH_TOKEN_URL,
        data=token_req_payload,
        allow_redirects=False,
        auth=(CLIENT_ID, CLIENT_SECRET),
    )
    token_response.raise_for_status()  # Raise an exception if the request failed
    tokens = json.loads(token_response.text)
    access_token = tokens["access_token"]

    # Open a client to the API
    client = Client.open(API_URL, headers={"Authorization": f"Bearer {access_token}"})

    return client

In [ ]:
'''
Function -- get_shapely_geometry
    Get shapely geometry and display the area on map according to the given coordinates
Parameters:
    coordinates -- a list of coordinates that form the area to be displayed
        e.g. [[lon1, lat1], [lon2, lat2], ...]
        generate at https://geojson.io/
Return:
    location -- a string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
    shapely_geometry -- a shapely geometry of the given coordinates
'''

def get_shapely_geometry(location, coordinates):
    geojson_geometry = {
        "type": "Polygon",
        "coordinates": [
            coordinates
        ],
    }

    # Convert the GeoJSON geometry into a shapely geometry
    shapely_geometry = shape(geojson_geometry)
    # Create a GeoDataFrame with the shapely geometry
    # Note that the Coordinate Reference System (CRS) defines how the two-dimensional,
    # projected geometry relates to real world locations.
    gdf = gpd.GeoDataFrame([{"geometry": shapely_geometry}], crs="EPSG:4326")

    # Calculate the centroid of your GeoDataFrame to center the map
    centroid = gdf.geometry.centroid.unary_union.centroid

    # Create a folium map centered on the centroid of your shapefile
    m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

    # Add the GeoDataFrame as a layer to the folium map
    folium.GeoJson(gdf, name="geojson").add_to(m)

    # Add layer control to toggle the geojson layer
    folium.LayerControl().add_to(m)

    # Display the map
    print("location of " + location + ":")
    m

    return shapely_geometry

In [ ]:
'''
Function -- get_sentinel2_data
    Download Sentinel-2 data from the Earth Platform.
Parameters:
    client -- client of the API
    aoi -- area of interest in shapely geometry
    start_date -- start date of the search
    end_date -- end date of the search
    cloud_cover -- cloud cover threshold
    max_items -- maximum number of items to be returned per page
Return:
    items -- a list of items that correspond to tiles found in the GeoDataFrame
    gdf -- a GeoDataFrame of items
'''
def get_sentinel2_data(
    client: Client,
    aoi: dict,
    start_date: str = "2019-06-01",
    end_date: str = "2019-09-30",
    cloud_cover: float = 3,
    max_items: int = 50,
    ):
    """
    Download Sentinel-2 data from the Earth Platform.
    """
    query = client.search(
        collections=["sentinel-2-l2a"],
        datetime=f"{start_date}T00:00:00.000000Z/{end_date}T00:00:00.000000Z",  # 2023-07-10T00:00:00.000000Z/2023-07-20T00:00:00.000000Z
        intersects=aoi,  # The area of interest; you can also query by bbox, or other geometry
        query={"eo:cloud_cover": {"lte": cloud_cover}},
        sortby=[
            {
                "field": "properties.eo:cloud_cover",
                "direction": "asc",
            },  # Sort by cloud cover from lowest to highest
        ],
        limit=max_items,  # This is the number of items to be returned per page
        max_items=max_items,  # This is number of items to page over
    )
    items = list(query.items())
    if len(items) == 0:
        raise Exception(
            "No items found, try enlarging search area or increasing cloud cover threshold."
        )
    print(f"Found: {len(items):d} tiles.")

    # Convert STAC items into a GeoJSON FeatureCollection
    stac_json = query.item_collection_as_dict()
    gdf = gpd.GeoDataFrame.from_features(stac_json, crs="EPSG:4326")

    return items, gdf

In [ ]:
'''
Function -- peek_sentinel_items
    Take a look at the metadata of the first sentinel items
    and display the indices and id of all the items
Parameters:
    items -- a list of items that correspond to tiles found in the GeoDataFrame
'''
def peek_sentinel_items(items):
    print("\nIndices and IDs of all the sentinel items:")
    counter = 0
    for item in items:
      print("Index ", counter, ": ", item.id)
      counter += 1

In [ ]:
'''
helper functions for the cell below
'''

def download_file(href: str, outpath: Path):
    """
    Given a URL, download the file to the specified path.
    """
    with requests.get(href, stream=True) as r:
        r.raise_for_status()
        with open(outpath, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

def download_files_for_item(
    item: Item, asset_dict: dict[str, str], outpath: Path, debug: bool = True
) -> bool:
    """
    Save all files of interest for a given item.

    If one file fails to download return False, otherwise return True.
    """

    for key, value in asset_dict.items():
        if debug:
          print(f"Downloading {key} and relabeling to {value}")
        if key in item.assets:
            if key in ["tileinfo_metadata"]:
                file_outpath = outpath / f"{value}.json"
            else:
                file_outpath = outpath / f"{value}.tiff"
            if not file_outpath.exists():
                try:
                    download_file(item.assets[key].href, file_outpath)
                except requests.ConnectionError:
                    print(
                        f"Failed to download {item.assets[key].href} for item {item.id}"
                    )
                    return False
                except requests.exceptions.ReadTimeout:
                    print(
                        f"Experienced a read timeout for {item.assets[key].href} for item {item.id}"
                    )
                    return False
            else:
                print(f"Skipping {item.assets[key].href} as it already exists.")

    return True

In [ ]:
'''
Function -- download_and_tile_files
    Given a GeoDataFrame of items and a list of STAC items, download the
    files to a output directory created by the given location name.
Parameters:
    items: A list of items that correspond to tiles found in the GeoDataFrame
    download_files: A dictionary of strings where the keys correspond to names of items on the Earth Platform
        and the values correspond to their Sentinel-2 name.
    location: A string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
'''
def download_and_tile_files(items: list[Item], download_files: dict[str, str], location: str):
    """
    Given a GeoDataFrame of items and a list of STAC items, download the
    files to a given output directory.

    Parameters:
      items: A list of items that correspond to tiles found in the GeoDataFrame and include paths to files to be
        downloaded in this function.
      download_files: A dictionary of strings where the keys correspond to names of items on the Earth Platform
        and the values correspond to their Sentinel-2 name.
    """
    # delete the folder if it already exists
    output_dir = Path(f"/content/sentinel_tiles")
    outpath = output_dir / location
    if os.path.exists(outpath):
        shutil.rmtree(outpath)
    outpath.mkdir(exist_ok=True, parents=True)

    downloaded = 0
    for index, tile in enumerate(items):
        downloaded = download_files_for_item(tile, download_files, outpath)

        if downloaded:
            downloaded += 1
        else:
            print(f"Unable to download file for item with id: {tile.id} at index: {index} in items list.")

    print(
        f"Downloaded all bands for {downloaded} tiles. Failed to download at least one "
        + f"band or file for {len(items) - downloaded} tiles."
    )

In [ ]:
'''
Function -- get_ground_truth
    open and resized the ground truth image
Parameters:
    location -- a string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
Return:
    resized_ground_truth -- a numpy array of the ground truth image
'''
def get_ground_truth(location: str):
    ground_truth_path = '/content/ground_truth/' + location + '.tif'
    ground_truth = Image.open(ground_truth_path)
    ground_truth_array = np.array(ground_truth)

    b04_size = tiff.imread(f"/content/sentinel_tiles/{location}/cropped/B04.tiff").shape

    resized_ground_truth = cv2.resize(ground_truth_array, b04_size[::-1])

    return resized_ground_truth

#### 1-2-2. Data Processing Functions

In [ ]:
'''
Function --  crop_and_visualize
    crop the .tiff files according to the given coordinates
    visualize the cropped .tiff files
Parameters:
    location -- a string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
    coordinates -- a list of coordinates that form the area to be displayed
        e.g. [[lon1, lat1], [lon2, lat2], ...]
        generate at https://geojson.io/
'''

def crop_and_visualize(location: str, coordinates):
    # Directory containing the .tiff files (update with your input path)
    input_directory = '/content/sentinel_tiles/' + location

    # Directory to save the cropped .tiff files
    output_directory = '/content/sentinel_tiles/' + location + '/cropped'

    # Ensure a new empty output directory exists
    if os.path.exists(output_directory):
        shutil.rmtree(output_directory)
    os.makedirs(output_directory)

    # List of bands to crop
    bands = [
        "B02.tiff", "B03.tiff", "B04.tiff", "B05.tiff", "B06.tiff", "B07.tiff",
        "B08.tiff", "B11.tiff", "B12.tiff", "B8A.tiff",
    ]

    for band in bands:
        input_path = os.path.join(input_directory, band)
        output_path = os.path.join(output_directory, band)

        with rasterio.open(input_path) as src:
            # Get the CRS of the image
            image_crs = src.crs

            # Create a transformer to convert from latitude/longitude to the image's CRS
            transformer = Transformer.from_crs("EPSG:4326", image_crs, always_xy=True)

            # Convert the polygon coordinates
            transformed_coords = [transformer.transform(lon, lat) for lon, lat in coordinates]

            # Create a polygon
            polygon = Polygon(transformed_coords)
            geojson_polygon = [mapping(polygon)]

            # Mask and crop the image using the polygon
            out_image, out_transform = mask(src, geojson_polygon, crop=True)

            # Update the metadata to reflect the new dimensions and transform
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            # Write the cropped data to a new file
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)

    print("Cropping complete!")

    # Loop through the bands and plot each cropped .tiff file
    for band_name in bands:
        file_path = f'{output_directory}/{band_name}'
        with rasterio.open(file_path) as src:
            band = src.read(1)
            plt.figure(figsize=(10, 10))
            plt.title(band_name)
            plt.imshow(band, cmap='viridis', norm=Normalize(vmin=band.min(), vmax=band.max()))
            plt.colorbar()
            plt.axis('off')
            plt.show()

In [ ]:
'''
helper functions for the cell below
'''

def display_images(image_dict):
    fig, axes = plt.subplots(2, 3, figsize=(24, 16))
    fig.tight_layout()

    for i, (title, image) in enumerate(image_dict.items()):
        row = i // 3
        col = i % 3
        axes[row, col].imshow(image)
        axes[row, col].set_title(title)
        axes[row, col].axis("off")

    plt.show()

def brighten(band: np.ndarray):
    """Adjust image brightness using an alpha value (0 to 1). Higher alpha increases brightness, lower decreases."""

    alpha=0.2 # you can change this value
    beta=0
    return np.clip(alpha*band+beta, 0,255)

def normalize(band: np.ndarray) -> np.ndarray:
    """
    First brighten the bands and then return normalized image
    """
    band = brighten(band)
    band_min, band_max = (band.min(), band.max())

    return ((band-band_min)/((band_max - band_min)))

In [ ]:
'''
Function -- generate_spectral_images
    generate RGB and spectral images from the cropped .tiff files
Parameters:
    location -- a string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
Return:
    spectral_images -- a dictionary of images that are NDVI, NDVI705, NDBI, NDWI, NBR, RGB
'''
def generate_spectral_images(location: str):
    data_path = '/content/sentinel_tiles/' + location + '/cropped'
    # Unpack the bands
    band_size = tiff.imread(f"{data_path}/B02.tiff").shape
    print(band_size)

    B02 = normalize(
            tiff.imread(f"{data_path}/B02.tiff"),)
    B03 = normalize(
            tiff.imread(f"{data_path}/B03.tiff"),)
    B04 = normalize(
            tiff.imread(f"{data_path}/B04.tiff"),)
    B05 = normalize(
            cv2.resize(
                tiff.imread(f"{data_path}/B05.tiff"),
                band_size[::-1]# make sure the dimensions of B11 is the same as the other bands after resizing
            ))
    B08 = normalize(
            tiff.imread(f"{data_path}/B08.tiff"),)

    B11 = normalize(
            cv2.resize(
                tiff.imread(f"{data_path}/B11.tiff"),
                band_size[::-1]# make sure the dimensions of B11 is the same as the other bands after resizing
            ))

    B12 = normalize(
            cv2.resize(
                tiff.imread(f"{data_path}/B12.tiff"),
                band_size[::-1]
            ))

    # Calculate NDVI (Normalized Difference Vegetation Index)
    NDVI = (B08 - B04) / (B08 + B04)

    # Calculate NDVI705
    NDVI705 = (B05 - B04) / (B05 + B04)

    # Calculate NDBI (Normalized Difference Built-Up Index)
    NDBI = (B11 - B08) / (B11 + B08)

    # # Calculate NDWI (Normalized Difference Water Index)
    NDWI = (B08 - B12) / (B08 + B12)

    # # Calculate NBR (Normalized Burn Ratio)
    NBR = (B08 - B12) / (B08 + B12)

    # Create a color image using RGB bands
    RGB = np.stack([B04, B03, B02], axis=-1)

    spectral_images = {
        "NDVI": NDVI,
        "NDVI705": NDVI705,
        "NDBI": NDBI,
        "NDWI": NDWI,
        "NBR": NBR,
        "RGB": RGB,
    }

    display_images(spectral_images)

    return spectral_images

#### 1-2-3. Method Applying Functions

In [ ]:
'''
texture extraction methods
'''

# convert the given rgb image to unit 8 bit image
def convert_rgb(rgb_image):
    converted_rgb = copy.deepcopy(rgb_image)
    converted_rgb *= 255
    converted_rgb = converted_rgb // 1
    converted_rgb = converted_rgb.astype(np.uint8)
    return converted_rgb

def laplacian(rgb_image, kernel_size = 5):
    greyscale = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    result = cv2.Laplacian(greyscale, cv2.CV_8U, ksize=kernel_size)  # kernel size may be affected by image resolution

    return result

# # Parameters for Gabor filter
# ksize -- Kernel size
# sigma -- Standard deviation of the Gaussian function
# theta -- Orientation of the normal to the parallel stripes
# lambd -- Wavelength of the sinusoidal factor
# gamma -- Spatial aspect ratio
# psi -- Phase offset
def gabor(rgb_image, ksize = 31, sigma = 4.0, theta = np.pi/4, lambd = 10.0, gamma = 0.5, psi = 0):
    greyscale = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    # Create Gabor kernel and apply filter
    gabor_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, ktype=cv2.CV_32F)
    result = cv2.filter2D(greyscale, cv2.CV_8UC3, gabor_kernel)

    return result

def sobel(rgb_image, kernel_size = 5):
    greyscale = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

    # Calculate the Sobel gradients in the x and y directions
    sobelx = cv2.Sobel(greyscale, cv2.CV_32F, 1, 0, ksize=kernel_size)
    sobely = cv2.Sobel(greyscale, cv2.CV_32F, 0, 1, ksize=kernel_size)

    # Combine the Sobel gradients to get the overall edge response
    sobel_combined = cv2.magnitude(sobelx, sobely)

    # Convert the result back to 8-bit for display purposes
    sobel_combined = cv2.convertScaleAbs(sobel_combined)

    return sobel_combined

def close(grayscale_image, kernel_size = (5, 5)):
    kernel = np.ones(kernel_size, np.uint8)
    result = cv2.morphologyEx(grayscale_image, cv2.MORPH_CLOSE, kernel)

    return result

In [ ]:
'''
Function -- apply_algorithm
    apply the algorithm, visualize the results, write the results to summary_dict
Parameters:
    location -- a string that is the name of the location connected with underscores e.g. 'garibaldi_lake'
    rgb -- a numpy array of the RGB image
    ndvi -- a numpy array of the NDVI image
    ground_truth -- a numpy array of the ground truth image
    texture_method -- the name of the chosen method to extract the texture value
        available options: 'laplacian', 'sobel', 'gabor'
    texture_threshold -- the threshold texture value for identifying forest
    ndvi_threshold -- the threshold NDVI value for identifying forest
    summary_dict -- a nested dictionary that stores the results all experiments
'''
def apply_algorithm(location: str, rgb, ndvi, ground_truth, texture_method, texture_threshold, ndvi_threshold, summary_dict):
    forest_colors_in_groundtruth = [[153, 51, 204], [0, 102, 0], [0, 204, 0], [204, 153,0]]
    truth = []
    prediction = []

    fig, axes = plt.subplots(2, 3, figsize=(24, 16))

    # ground truth
    axes[0][0].imshow(ground_truth)
    axes[0][0].set_title("Ground Truth", fontsize=30)
    axes[0][0].axis("off")

    rgb = convert_rgb(rgb)
    # texture
    if texture_method == 'sobel':
        texture = sobel(rgb)
        axes[0][1].imshow(texture, cmap='gray')
        axes[0][1].set_title("sobel", fontsize=30)
        axes[0][1].axis("off")
    else:
      if texture_method == 'laplacian':
          texture = laplacian(rgb)
          texture = close(texture)
          axes[0][1].imshow(texture, cmap='gray')
          axes[0][1].set_title("laplacian+closing", fontsize=30)
          axes[0][1].axis("off")
      elif texture_method == 'gabor':
          texture = gabor(rgb)
          texture = close(texture)
          axes[0][1].imshow(texture, cmap='gray')
          axes[0][1].set_title("gabor+closing", fontsize=30)
          axes[0][1].axis("off")

    # ndvi
    axes[0][2].imshow(ndvi)
    axes[0][2].set_title("NDVI", fontsize=30)
    axes[0][2].axis("off")

    # identification
    identification = rgb.copy()

    false_positive = rgb.copy()
    false_positive_overlay = np.zeros_like(rgb, dtype=np.uint8)

    false_negative = rgb.copy()
    false_negative_overlay = np.zeros_like(rgb, dtype=np.uint8)

    for i in range(rgb.shape[0]):
        for j in range(rgb.shape[1]):
            if texture[i][j] > texture_threshold and ndvi[i][j] > ndvi_threshold:
                identification[i][j] = [255, 0, 0]
                prediction.append(1)
            else:
                prediction.append(0)

            if [ground_truth[i][j][0], ground_truth[i][j][1], ground_truth[i][j][2]] in forest_colors_in_groundtruth:
                truth.append(1)
            else:
                truth.append(0)

            if prediction[-1] == 0 and truth[-1] == 1:
                false_negative_overlay[i][j] = [255, 0, 0]
            elif prediction[-1] == 1 and truth[-1] == 0:
                false_positive_overlay[i][j] = [255, 0, 0]

    alpha = 0.7
    false_positive = cv2.addWeighted(false_positive_overlay, alpha, false_positive, 1, 0)
    false_negative = cv2.addWeighted(false_negative_overlay, alpha, false_negative, 1, 0)

    axes[1][0].imshow(identification)
    axes[1][0].set_title("Forest Identification", fontsize=30)
    axes[1][0].axis("off")

    axes[1][1].imshow(false_positive)
    axes[1][1].set_title("False Positive", fontsize=30)
    axes[1][1].axis("off")

    axes[1][2].imshow(false_negative)
    axes[1][2].set_title("False Negative", fontsize=30)
    axes[1][2].axis("off")

    plt.show()

    # confusion matrix
    cmatrix = confusion_matrix(truth, prediction)
    confusion_matrix_display = ConfusionMatrixDisplay(confusion_matrix=cmatrix, display_labels=["Non-Forest", "Forest"])
    confusion_matrix_display.plot(values_format='', cmap='Greens')
    plt.title("Confusion Matrix for Forest Identification")
    plt.show()

    # metrics: accuracy, precision, recall, f1, kappa
    # visualize and save to summary_dict
    accuracy = accuracy_score(truth, prediction)
    precision = precision_score(truth, prediction)
    recall = recall_score(truth, prediction)
    f1 = f1_score(truth, prediction)
    kappa = cohen_kappa_score(truth, prediction)

    summary_dict[location][texture_method]['accuracy'] = accuracy
    summary_dict[location][texture_method]['precision'] = precision
    summary_dict[location][texture_method]['recall'] = recall
    summary_dict[location][texture_method]['f1'] = f1
    summary_dict[location][texture_method]['kappa'] = kappa

    data = {
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'],
        'Value': [accuracy, precision, recall, f1, kappa]
    }
    df = pd.DataFrame(data)

    fig, ax = plt.subplots(figsize=(6, 2))
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    plt.title("Evaluation Metrics", fontsize=16, pad=20)

    plt.show()

In [ ]:
'''
Function -- visualize_summary_dict
    visualize the summary_dict with a table and a curve chart for each location
'''

def visualize_summary_dict(summary_dict):
    # Iterate over each location in the summary_dict
    for location in summary_dict:
        methods = list(summary_dict[location].keys())
        metrics = list(summary_dict[location][methods[0]].keys())

        # Create a DataFrame to store the metrics for the current location
        data = {method: [summary_dict[location][method][metric] for metric in metrics] for method in methods}
        df = pd.DataFrame(data, index=metrics).T  # Transpose so that metrics are columns

        # Create a one-row-two-column plot
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

        # Plot the table
        ax1.axis('tight')
        ax1.axis('off')
        table = ax1.table(cellText=df.values, colLabels=metrics, rowLabels=methods, cellLoc='center', loc='center')
        ax1.set_title(f"Metrics Table for {location}", fontsize=16, pad=20)

        # Plot the curve chart
        for method in methods:
            ax2.plot(metrics, df.loc[method], marker='o', label=method)

        ax2.set_title(f"Metrics Curves for {location}", fontsize=16)
        ax2.set_xlabel("Metrics")
        ax2.set_ylabel("Values")
        ax2.legend(title="Methods")
        ax2.grid(True)

        # Display the plot
        plt.tight_layout()
        plt.show()

## 2. Preparation

Before Everything: Download the 'ground_truth' folder from [GitHub Repository](https://github.com/kaycaimx/GreenGuardians/tree/main/ground_truth). Move it and the 'secrets.env' to the current directory.

### 2-1. Global Variables

In [ ]:
# thresholds
laplacian_threshold = 64
gabor_threshold = 64
sobel_threshold = 80
ndvi_threshold = 0.585

In [ ]:
# api client
client = get_api_client()

In [ ]:
# if the cell is rerun, rerunning all cells of "3. Experiment" is needed

# summary dictionary
summary_dict = nested_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(float)))

In [ ]:
# location to coordinates dictionary
location_to_coordinates = {
    'garibaldi_lake': [[-123.11679830457658,49.96048123499753],
                       [-123.11679830457658,49.89344994731354],
                       [-122.98558891107739,49.89344994731354],
                       [-122.98558891107739,49.96048123499753],
                       [-123.11679830457658,49.96048123499753]],
    'joffre_lake': [[-122.50604002994187, 50.37180675672768],
                    [-122.50604002994187, 50.34267450253444],
                    [-122.46990530921299, 50.34267450253444],
                    [-122.46990530921299, 50.37180675672768],
                    [-122.50604002994187, 50.37180675672768]],
    'pitt_meadows': [[-122.710543, 49.359739],
                     [-122.710543, 49.293932],
                     [-122.513177, 49.293932],
                     [-122.513177, 49.359739],
                     [-122.710543, 49.359739]],
    'squamish_river': [[-123.265109, 49.733304],
                       [-123.265109, 49.682254],
                       [-123.130048, 49.682254],
                       [-123.130048, 49.733304],
                       [-123.265109, 49.733304]],
    'magnum_mine': [[-125.22773492695391, 58.61666627427516],
                    [-125.22773492695391, 58.505103068939604],
                    [-124.92201156470674, 58.505103068939604],
                    [-124.92201156470674, 58.61666627427516],
                    [-125.22773492695391, 58.61666627427516]],
    'okanagan': [[-119.41802379338398, 50.09514353328319],
                 [-119.41802379338398, 50.044896274137415],
                 [-119.30277738682935, 50.044896274137415],
                 [-119.30277738682935, 50.09514353328319],
                 [-119.41802379338398, 50.09514353328319]],
    'vancouver_island': [[-128.11000410456677, 50.828615612651106],
                         [-128.11000410456677, 50.76487417731673],
                         [-127.96286271998287, 50.76487417731673],
                         [-127.96286271998287, 50.828615612651106],
                         [-128.11000410456677, 50.828615612651106]],
    'abbotsford': [[-122.08909459981263, 49.09838991642172],
            [-122.08909459981263, 49.02657685560854],
            [-121.9538837206437, 49.02657685560854],
            [-121.9538837206437, 49.09838991642172],
            [-122.08909459981263, 49.09838991642172]],
    'prince_george': [[-122.80737010729996, 53.947719249562965],
            [-122.80737010729996, 53.886832581763144],
            [-122.67207140284724, 53.886832581763144],
            [-122.67207140284724, 53.947719249562965],
            [-122.80737010729996, 53.947719249562965]]
}

In [ ]:
# set files to download
high_resolution_bands = {"red": "B04", "green": "B03", "blue": "B02", "nir": "B08"}

mid_resolution_bands = {
    "rededge1": "B05",
    "rededge2": "B06",
    "rededge3": "B07",
    "nir08": "B8A",
    "swir16": "B11",
    "swir22": "B12",
}

all_download_files = {
    **high_resolution_bands,
    **mid_resolution_bands,
}

### 2-2. Convert Coordinate
convert coordinates to shapely geometry<br>
visualize the locations for check

In [ ]:
location_to_shapely_geometry = {}

for location, coordinates in location_to_coordinates.items():
    location_to_shapely_geometry[location] = get_shapely_geometry(location, coordinates)

### 2-3. Peek Sentinel Data
print basic infomation of the sentinel data for each location

In [ ]:
for location, aoi in location_to_shapely_geometry.items():
  print('='*30)
  print("Sentinel data of " + location)
  print('='*30)
  sentinel_items, _ = get_sentinel2_data(client, aoi)
  peek_sentinel_items(sentinel_items)
  print('\n'*2)

### 2-4. Download Sentinel Data
refer to the print results of "2-3. Peek Sentinel Data" when choosing item index

#### 2-4-1. Garibaldi Lake

In [ ]:
item_index = 6
location = 'garibaldi_lake'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-2. Joffre Lake

In [ ]:
item_index = 1
location = 'joffre_lake'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

In [ ]:
item_index = 6
location = 'garibaldi_lake'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-3. Pitt Meadows

In [ ]:
item_index = 0
location = 'pitt_meadows'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-4. Squamish River

In [ ]:
item_index = 1
location = 'squamish_river'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-5. Magnum Mine

In [ ]:
item_index = 4
location = 'magnum_mine'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-6. okanagan

In [ ]:
item_index = 1
location = 'okanagan'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-7. Vancouver Island

In [ ]:
item_index = 1
location = 'vancouver_island'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-8. Abbotsford

In [ ]:
item_index = 0
location = 'abbotsford'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

#### 2-4-9. Prince George

In [ ]:
item_index = 0
location = 'prince_george'

In [ ]:
# download
sentinel_items, _ = get_sentinel2_data(client, location_to_shapely_geometry[location])
item = sentinel_items[item_index:item_index + 1]
download_and_tile_files(item, all_download_files, location)

#crop
crop_and_visualize(location, location_to_coordinates[location])

## 3. Experiment

subsections share variable names, therefore, run each section in a whole is needed

### 3-1. Garibaldi Lake

In [ ]:
#location
location = 'garibaldi_lake'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-2. Joffre Lake

In [ ]:
#location
location = 'joffre_lake'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-3. Pitt Meadows

In [ ]:
#location
location = 'pitt_meadows'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-4. Squamish River

In [ ]:
#location
location = 'squamish_river'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-5. Magnum Mine

In [ ]:
#location
location = 'magnum_mine'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-6. Okanagan

In [ ]:
#location
location = 'okanagan'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-7. Vancouver Island

In [ ]:
#location
location = 'vancouver_island'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-8. Abbotsford

In [ ]:
#location
location = 'abbotsford'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

### 3-9. Prince George

In [ ]:
#location
location = 'prince_george'

# ground truth
ground_truth = get_ground_truth(location)

In [ ]:
# generate and visualize spectral and rgb
spectral_images = generate_spectral_images(location)

In [ ]:
'''
Laplacian + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'laplacian', laplacian_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Gabor + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'gabor', gabor_threshold, ndvi_threshold, summary_dict)

In [ ]:
'''
Sobel + NDVI
'''
apply_algorithm(location, spectral_images['RGB'], spectral_images['NDVI'], ground_truth, 'sobel', sobel_threshold, ndvi_threshold, summary_dict)

## 4. Result Summary

In [ ]:
visualize_summary_dict(summary_dict)

## 5. Other

### 5-1. Ground Truth Generating
Visit [this](https://code.earthengine.google.com/?scriptPath=users/sat-io/awesome-gee-catalog-examples:agriculture-vegetation-forestry/CA-FORESTED-ECOSYSTEM-LC). Replace the code with the code below.

In [ ]:
# // Ground truth download
# var ca_lc_last = ee.Image(ca_lc.sort('system:time_start',false).first());

# var from = [0, 20, 31, 32, 33, 40, 50, 80, 81, 100, 210, 220, 230];
# var to =   [0, 1,  2,  3,  4,  5,  6,  7,  8,  9,   10,  11,  12 ];
# ca_lc_last = ca_lc_last.remap(from, to);

# print("Reclassed values:");
# print({"from": from, "to": to});

# // Define a dictionary which will be used to make legend and visualize image on map
# var dict = {
#   "names": [
#   "Unclassified",
#   "Water",
#   "Snow/Ice",
#   "Rock/Rubble",
#   "Exposed/Barren land",
#   "Bryoids",
#   "Shrubs",
#   "Wetland",
#   "Wetland-treed",
#   "Herbs",
#   "Coniferous",
#   "Broadleaf",
#   "Mixedwood"
#   ],
#   "colors": [
#     "#686868",
#     "#3333ff",
#     "#ccffff",
#     "#cccccc",
#     "#996633",
#     "#ffccff",
#     "#ffff00",
#     "#993399",
#     "#9933cc",
#     "#ccff33",
#     "#006600",
#     "#00cc00",
#     "#cc9900"
#   ]};

# // Create a panel to hold the legend widget
# var legend = ui.Panel({
#   style: {
#     position: 'bottom-left',
#     padding: '8px 15px'
#   }
# });

# // Function to generate the legend
# function addCategoricalLegend(panel, dict, title) {

#   // Create and add the legend title.
#   var legendTitle = ui.Label({
#     value: title,
#     style: {
#       fontWeight: 'bold',
#       fontSize: '18px',
#       margin: '0 0 4px 0',
#       padding: '0'
#     }
#   });
#   panel.add(legendTitle);

#   var loading = ui.Label('Loading legend...', {margin: '2px 0 4px 0'});
#   panel.add(loading);

#   // Creates and styles 1 row of the legend.
#   var makeRow = function(color, name) {
#     // Create the label that is actually the colored box.
#     var colorBox = ui.Label({
#       style: {
#         backgroundColor: color,
#         // Use padding to give the box height and width.
#         padding: '8px',
#         margin: '0 0 4px 0'
#       }
#     });

#     // Create the label filled with the description text.
#     var description = ui.Label({
#       value: name,
#       style: {margin: '0 0 4px 6px'}
#     });

#     return ui.Panel({
#       widgets: [colorBox, description],
#       layout: ui.Panel.Layout.Flow('horizontal')
#     });
#   };

#   // Get the list of palette colors and class names from the image.
#   var palette = dict['colors'];
#   var names = dict['names'];
#   loading.style().set('shown', false);

#   for (var i = 0; i < names.length; i++) {
#     panel.add(makeRow(palette[i], names[i]));
#   }

#   Map.add(panel);

# }


# /*
#   // Display map and legend ///////////////////////////////////////////////////////////////////////////////
# */

# // Add the legend to the map
# addCategoricalLegend(legend, dict, 'CA Annual forest LC map 2019');

# Map.setCenter(-97.61655457157725,55.6280720462063,4)

# // Add image to the map
# Map.addLayer(ca_lc_last.mask(ca_lc_last.neq(0)), {min:0, max:12, palette:dict['colors']}, 'CA Annual forest LC map 2019')

# // Add a RGB layer
# // Define the visualization parameters with the color palette
# var visualization = {
#   min: 0,
#   max: 12,
#   palette: [
#     "#686868", "#3333ff", "#ccffff", "#cccccc", "#996633",
#     "#ffccff", "#ffff00", "#993399", "#9933cc", "#ccff33",
#     "#006600", "#00cc00", "#cc9900"
#   ]
# };

# // Apply the visualization parameters
# var visualizedImage = ca_lc_last.visualize(visualization);

# // Display the RGB image on the map
# Map.addLayer(visualizedImage, {}, 'CA Annual Forest LC Map 2019 RGB');

# // Download a specific area
# var geojsonGeometry = {
#   "type": "Polygon",
#   "coordinates": [
#     [
#       [
#               -123.11679830457658, 49.96048123499753
#             ],
#             [
#               -123.11679830457658, 49.89344994731354
#             ],
#             [
#               -122.98558891107739, 49.89344994731354
#             ],
#             [
#               -122.98558891107739, 49.96048123499753
#             ],
#     ]
#   ]
# };

# // Create an Earth Engine geometry object using GeoJSON
# var region = ee.Geometry(geojsonGeometry);

# Map.centerObject(region, 15); // Center the map on this area at a higher zoom level
# Map.addLayer(region, {color: 'red'}, 'Custom Polygon Region');

# // Export.image.toDrive({
# //   image: ca_lc_last, // name of the image
# //   description: 'stanley_park_full',
# //   scale: 30,
# //   region: region  // Use the defined polygon area
# // });

# // Export the RGB image
# Export.image.toDrive({
#   image: visualizedImage,
#   description: 'garibaldi_lake_RGB',
#   scale: 30,
#   region: region,  // Use the defined polygon area
#   formatOptions: {
#     cloudOptimized: true
#   }
# });

# //Download setting --> EPSG:32610